In [2]:
import habitat
import numpy as np
import matplotlib.pyplot as plt
from habitat.tasks.nav.shortest_path_follower import ShortestPathFollower
from PIL import Image
import uuid
import os
import shutil
from typing import Any
import quaternion
import random
from env import Env
import matplotlib.colors as mcolors
from utils import draw_top_down_map, AgentPosition
from matplotlib.colors import LinearSegmentedColormap
import matplotlib

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
config = habitat.get_config("/data_config/challenge_objectnav2020.local.rgbd.yaml")

config.defrost()

config.DATASET.DATA_PATH = '/data/datasets/objectnav/mp3d/v1/{split}/{split}.json.gz'
config.DATASET.SCENES_DIR = '/data/scene_datasets'
config.DATASET.SPLIT = "val_mini"

config.SIMULATOR.HABITAT_SIM_V0.ALLOW_SLIDING = True
#config.SIMULATOR.RGB_SENSOR.NOISE_MODEL_KWARGS.intensity_constant = 0.01
#config.SIMULATOR.NOISE_MODEL.NOISE_MULTIPLIER = 0.0

H = 320
W = 640

config.SIMULATOR.RGB_SENSOR.HEIGHT = H
config.SIMULATOR.RGB_SENSOR.WIDTH = W
config.SIMULATOR.DEPTH_SENSOR.HEIGHT = H
config.SIMULATOR.DEPTH_SENSOR.WIDTH = W
config.SIMULATOR.SEMANTIC_SENSOR.HEIGHT = H
config.SIMULATOR.SEMANTIC_SENSOR.WIDTH = W

config.PYROBOT.RGB_SENSOR.HEIGHT = H
config.PYROBOT.RGB_SENSOR.WIDTH = W
config.PYROBOT.DEPTH_SENSOR.HEIGHT = H
config.PYROBOT.DEPTH_SENSOR.WIDTH = W

config.SIMULATOR.HABITAT_SIM_V0.ALLOW_SLIDING = True

config.SIMULATOR.AGENT_0.SENSORS.append('SEMANTIC_SENSOR')

config.SIMULATOR.TURN_ANGLE = 5
config.SIMULATOR.TILT_ANGLE = 5
config.SIMULATOR.FORWARD_STEP_SIZE = 0.15

config.TASK.MEASUREMENTS.append("TOP_DOWN_MAP")
config.TASK.SENSORS.append("HEADING_SENSOR")
config.TASK.SENSORS.append("GPS_SENSOR")
config.TASK.GPS_SENSOR.DIMENSIONALITY = 2
config.TASK.GPS_SENSOR.GOAL_FORMAT = "CARTESIAN"

config.TASK.AGENT_POSITION_SENSOR = habitat.Config()
config.TASK.AGENT_POSITION_SENSOR.TYPE = "my_supercool_sensor"
config.TASK.AGENT_POSITION_SENSOR.ANSWER_TO_LIFE = 42
config.TASK.SENSORS.append("AGENT_POSITION_SENSOR")

config.freeze()

In [5]:
env = Env(config)

2020-07-16 10:30:47,545 Initializing dataset ObjectNav-v1
Initializing dataset ObjectNav-v1
2020-07-16 10:30:47,663 initializing sim Sim-v0
initializing sim Sim-v0
2020-07-16 10:30:52,131 Initializing task ObjectNav-v1
Initializing task ObjectNav-v1
The answer to life is 42


In [8]:
observations, info, done = env.reset()

In [9]:
rgb = []
depth = []
semantic = []
objectgoal = []
compass = []
gps = []
heading = []
top_down_map = []
fog_of_war_mask = []
agent_map_coord = []
agent_angle = []
mapping = None
index_to_title_map = None

i = 0
while not done:
    i+=1
    observations, info, done = env.step(2,ideal_action=True)
    scene = env.sim.semantic_annotations()
    instance_id_to_label_id = {int(obj.id.split("_")[-1]): obj.category.index() for obj in scene.objects}
    mapping_ = np.array([ instance_id_to_label_id[i] for i in range(len(instance_id_to_label_id)) ])
    index_to_title_map_ = {obj.category.index(): obj.category.name() for obj in scene.objects }
    
    
    rgb.append(observations['rgb'])
    depth.append(observations['depth'])
    semantic.append(observations['semantic'])
    objectgoal.append(observations['objectgoal'])
    compass.append(observations['compass'])
    gps.append(observations['gps'])
    heading.append(observations['heading'])
    top_down_map.append(info['top_down_map']['map'])
    fog_of_war_mask.append(info['top_down_map']['fog_of_war_mask'])
    agent_map_coord.append(info['top_down_map']['agent_map_coord'])
    agent_angle.append(info['top_down_map']['agent_angle'])
    mapping = mapping_
    index_to_title_map = str(index_to_title_map_)
print(i)    

74


In [10]:
index_to_title_map

"{1: 'wall', 40: 'misc', 16: 'stairs', 4: 'door', 2: 'floor', 39: 'objects', 17: 'ceiling', 0: 'void', 14: 'plant', 24: 'column', -1: '', 5: 'table', 21: 'mirror', 15: 'sink', 7: 'cabinet', 28: 'lighting', 34: 'seating', 20: 'towel', 12: 'curtain', 9: 'window', 23: 'shower', 38: 'clothes', 11: 'bed', 35: 'board_panel', 37: 'appliances', 31: 'shelving', 26: 'counter', 22: 'tv_monitor', 3: 'chair'}"

In [11]:
import h5py

with h5py.File('data2.hdf5', 'w') as f:
    f.create_dataset("rgb", data=np.array(rgb))
    f.create_dataset("depth", data=np.array(depth))
    f.create_dataset("semantic", data=np.array(semantic))
    f.create_dataset("objectgoal", data=np.array(objectgoal))
    f.create_dataset("compass", data=np.array(compass))
    f.create_dataset("gps", data=np.array(gps))
    f.create_dataset("heading", data=np.array(heading))
    f.create_dataset("top_down_map", data=np.array(top_down_map))
    f.create_dataset("fog_of_war_mask", data=np.array(fog_of_war_mask))
    f.create_dataset("agent_map_coord", data=np.array(agent_map_coord))
    f.create_dataset("agent_angle", data=np.array(agent_angle))
    f.create_dataset("mapping", data=np.array(mapping))
    f.create_dataset("index_to_title_map", data=np.string_(index_to_title_map))